# Credit Risk Resampling Techniques

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt

In [4]:
# sklearn dependencies
from sklearn.model_selection import train_test_split
from skleran.preprocessing import StandardScalar
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay

ModuleNotFoundError: No module named 'skleran'

In [ ]:
# imbalanced-learning dependencies
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN

# Read the CSV and Perform Basic Data Cleaning

In [ ]:
#read csv 
file_path = Path('./Data/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df

In [ ]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [ ]:
# Load the data
file_path = Path('./Data/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
loans_df = df.loc[:, columns].copy()
loans_df.info()

In [ ]:
# Drop the null columns where all values are null
df = loans_df.dropna(axis='columns', how='all')
df.head()

In [ ]:
# Drop the null rows
df = loans_df.dropna()
df.info()

In [ ]:
loans_df.['loan_status'].value_counts()

In [ ]:
# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
loans_df = loans_df.loc[issued_mask]
loans_df['loan_status'].value_counts()

In [ ]:
# convert interest rate to numerical
loans_df['int_rate'] = loans_df['int_rate'].str.replace('%', '')
loans_df['int_rate'] = loans_df['int_rate'].astype('float') / 100

In [ ]:
# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
loans_df = loans_df.replace(x)
loans_df['loan_status'].value_counts

In [ ]:
x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
loans_df = loans_df.replace(x)
loans_df['loan_status']. value_counts()

In [ ]:
loans_df.reset_index(inplace=True, drop=True)

loans_df.head()

In [ ]:
# what are our categorical value columns
columns_obj = loans_df.select_dtypes(include = 'object').columns
print(len(columns_obj))

In [ ]:
# get dummies method in pandas

In [ ]:
# examine home_ownership
loans_df['home_ownership'].value_counts()

In [ ]:
# examine all the columns
for column in loans_df[columns_obj]:
    print(column, "\n", loans_df[column].value counts())

In [ ]:
# drop those columns that have just one value in value_counts()
loans_df = loans_df.drop(columns = ['debt_settlement_flag', 'hardship_flag', 'pymnt_plan'])
loans_df.shape

In [ ]:
# save our loans_df as clean_loans_df.csv file
cleans_loans = loans_df

# df.to_csv(index=False)
clean_loans_df.to_csv('./Data/clean_loans.csv",index=False)

In [ ]:
#read in the csv file
df = pd.read_csv("./Data/clean_loans.csv")
df.head()

# Split the Data into Training and Testing

In [ ]:
# Create our features
X = df.drop(columns = 'loan_status')
X

In [ ]:
# Create our target
y = df['loan_status']
y.value_counts()

In [1]:
# this is our imbalanced problem
print(347/68470)

0.005067912954578648


In [ ]:
X.describe()

In [ ]:
# use get_dummies to transform object dtypes into int
X_encoded = pd.get_dummies(X)
X_encoded.shape

In [ ]:
# all our columns in our features matrix (X) are numeric
X_encoded.describe()

In [ ]:
# Check the balance of our target values
y.shape

In [ ]:
# Check the balance of our target values
y['loan_status'].value_counts()

In [ ]:
### Use module 17.6.4 for standard-scaling X_encoded step ###

In [ ]:
# scale X_encoded feature matrix
data_scaler = StandardScaler()

In [ ]:
# fit and transform our X_encoded
X_encoded_scaled = data_scaler.fit_transform(X_encoded_df)
X_encoded_scaled[:5]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_encoded_scaled, y, random_state = 1)

In [ ]:
y_train.value_counts()

In [2]:
print(246/51366)

0.004789160144842892


In [ ]:
# Use Logical Regression to get performance baseline
# Calculate model accuracy score
# Generate confusion matrix
# Print out the classification reporting logistics regression
# Print out the imbalanced classification reporting logistics regression

# model
# fit
# assess

In [ ]:
# Instantiate Logistic Regression Model
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier

In [ ]:
Logistic Regression(random_state=1)

In [ ]:
# fit model with data (training)
classifier.fit(X_train, y_train)

In [ ]:
Logistic Regression(random_state=1)

In [ ]:
# create predictions using predict() method
y_pred = classifier.predict(X_test)
y_pred

In [ ]:
# view the data
pd.DataFrame({"Prediction": y_pred, "Actual": y_test})

In [ ]:
# calculate model accuracy
accuracy_score(y_test, Y_pred)

In [ ]:
# create confusion matrix
c_matrix = confusion_matrix(y_test, y_pred)
print(c_matrix)

In [ ]:
# display confusion matrix
c_matrix = confustion_matrix(y_test, y_pred, labels=clf.classes_)

In [ ]:
# confusion matrix using ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay(confusion_matrix=c_matrix, display_labels = clf.classes_)
disp.plot()

In [ ]:
# view classification report
report = classification_report(y_test, y_pred)
print(report)

In [ ]:
# view the imbalanced classification report
report = classification_report_imbalanced(y_test, y_pred)
print(report)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [ ]:
# Resample the training data with the RandomOverSampler
ros = RandomOverSampler(random=states1)
X_resampled, y_sampled = ros.fit_resample(X_train, y_train)

In [ ]:
# what is the "balance" or mix between Loans_status
Counter(y_sampled)

In [ ]:
# Train the Logistic Regression model using the resampled dat
# from sklearn.linear_model import Logistic Regression
# model
clf = LogisticRegression(solver='lbfgs', random_state=1)
# fit
clf.fit(X_resampled, y_resampled)
# predict
y_pred = clf.predict(X_test)

In [ ]:
# Calculated the balanced accuracy score
balanced_accuracy_score = balanced_accuracy_score(X_resampled, y_resampled)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

### SMOTE Oversampling

In [ ]:
# Resample the training data with SMOTE
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train, y_train)

In [ ]:
# Train the Logistic Regression model using the resampled data
clf = LogisticRegression(solver='lbfgs', random_state=1)
clf.fit(X_resampled, y_resampled)

In [ ]:
# check the balance of Loan_status
Counter(y_resampled)

Counter(0:451, 1:451)

In [ ]:
# model
clf = LogisticRegression(solver='lbfgs', random_state=1)
#fit
clf.fit(X_resampled, y_resampled)
#predict
y_pred = clf.predict(X_test)

In [ ]:
# Calculated the balanced accuracy score
y_pred = clf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
c_matrix = confusion_matrix(y_test, y_pred)

In [ ]:
# Display confusion matrix display
disp = ConfusionMatrixDisplay(confusion_matrix=c_matrix,display_label)
disp.plot()

# Save confusion display matrix image
plt.savefig("./Images/SMOTEENN_confusion_matrix_png")
plt.show()

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [ ]:
# Resample the data using the ClusterCentroids resampler
ros = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = ros.fit.resample(X_train, y_train)
Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE

# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [ ]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
# YOUR CODE HERE

In [ ]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE